# TITLE: Movie Recommendation system
## Collaborators
1. Ezra Kipchirchir
2. Sharon Kaliku
3. Mercy Tegekson
4. Robinson Mumo
5. Allen Maina
6. Candy Gudda

### Project Overview



### Introduction



### Challenges




### Proposed solutions



### Problem statement



### Data understanding


#### 1. Importing the required libraries and modules for our project

In [1]:
# importing modules
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns


#### 1.2 loading and doing the necessary inspection on our data

In [ ]:
pd.